In [162]:
import pandas as pd
from pandas import DataFrame as df
import xml.etree.ElementTree as ET

In [163]:
buy_out = df(columns=["date",
                        "phone_number",
                        "notebook",
                        "tablet",
                        "phone",
                        "spare_part",
                        "conversation",
                        "customer_price",
                        "our_price",
                        "recycle",
                        "customer_decision"])

map_facts_to_columns = dict()
map_facts_to_columns["phone_number"] = ["CustomerPhone"]
map_facts_to_columns["notebook"] = ["Word", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["phone"] = ["Phone", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["tablet"] = ["Tablet", "Vendor", "Model", "UserDefinedName"]
map_facts_to_columns["conversation"] = ["Buy"]
map_facts_to_columns["recycle"] = ["source sentence"]
map_facts_to_columns["customer_decision"] = ["what_customer_expects"]

buy_out

,date,phone_number,notebook,tablet,phone,spare_part,conversation,customer_price,our_price,recycle,customer_decision


In [164]:
tree = ET.parse('output.xml')
root = tree.getroot()
r = root.findall("Lead")

def make_dict_of_leads(root=root):
    dict_of_leads = dict();
    for lead in root.iter('Lead'):
        id = lead.attrib["id"]

        pulpy = ET.fromstring(lead.attrib["text"])
        source_sentence = pulpy.find("b").find("s");
        text = "";
        for i in source_sentence.itertext():
            text += i;
        dict_of_leads[id] = text;
    return dict_of_leads;

make_dict_of_leads()["46"]

'89140580517 продажа айфон6s на выезде Ляпидевского 12.'

In [195]:
# todo: помнить о тексте лида. Там выделены факты прямо в разметке - полезно при выводе информации в веб-интерфейсе

def compare_facts_to_leads(root=root):
    facts_grouped_by_lead = dict()

    for i in root.find("document").find('facts'):
        lead_id = i.attrib['LeadID']
        if facts_grouped_by_lead.get(lead_id):
            facts_grouped_by_lead[lead_id].append(i)
        else:
            facts_grouped_by_lead[lead_id] = [i]
    return facts_grouped_by_lead

calls = df(columns=[
        "CustomerPhone_Phone",
        "CustomerPlace_Place",
        "Buy_Word",
        "Pawn_Word",
        "Notebook_Word",
        "Notebook_Vendor",
        "Notebook_Model",
        "Notebook_UserDefinedName",
        "Phone_Word",
        "Phone_Vendor",
        "Phone_Model",
        "Phone_UserDefinedName",
        "source_sentence"
    ])

facts = compare_facts_to_leads()
leads = make_dict_of_leads()
for lead in facts:
    elems = facts[lead]
    one_sentence = lead
    cols = []
    values = []
    for fact_name in elems:
        for fact_field in fact_name:
            cols.append(fact_name.tag + "_" + fact_field.tag)
            values.append(fact_field.attrib["val"])
        
#     print(lead, values, cols)
    one_row = pd.DataFrame([values], columns=cols)
    calls = calls.append(one_row)

    values = []
    cols = []

    
calls.to_excel("whole_table.xlsx")
pawn = calls[calls.Pawn_Word.notnull()]
pawn.to_excel("panw_")

,Buy_NumberInPrice,Buy_Word,CustomerPhone_Phone,CustomerPlace_Place,Notebook_Model,Notebook_UserDefinedName,Notebook_Vendor,Notebook_Word,Pawn_Word,Phone_Model,Phone_UserDefinedName,Phone_Vendor,Phone_Word,source_sentence
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PAWN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,89140580517,NaN,NaN,NaN,АСЕР,НОУТБУК,PAWN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,89140580517,NaN,NaN,NaN,АСЕР,NaN,PAWN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,89140580517,NaN,NaN,NaN,NaN,NaN,PAWN,ХПЕРИЯ З3,NaN,СОНИ,NaN,NaN
0,NaN,NaN,89140580517,NaN,NaN,NaN,NaN,NaN,PAWN,ХПЕРИЯ З3,NaN,СОНИ,ТЕЛЕФОН,NaN
0,NaN,NaN,89140580517,NaN,NaN,NaN,NaN,НОУТБУК,PAWN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,89140580517,NaN,NaN,NaN,NaN,NaN,PAWN,ХПЕРИЯ ХПЕРИЯ З3,NaN,СОНИ,NaN,NaN
